In [1]:
# import os
# os.system("pip install --upgrade pip")
# os.system("python3.8 -m pip install --upgrade pip")
# os.system("pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis --upgrade")
# os.system("pip install pycelonis")

In [2]:
from pycelonis import *
from pycelonis.pql import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [3]:
url = "https://academic-if21b054-technikum-wien-at.eu-2.celonis.cloud/"

file = open('INNO_ProcessMining/Machine_Learning/APP_KEY.txt')
api_token = file.read()
file.close()

key_type = "APP_KEY"
celonis = get_celonis(base_url=url, api_token=api_token, key_type=key_type, permissions=False)

[2023-04-23 10:26:37,185] INFO: Initial connect successful! PyCelonis Version: 2.0.3


In [4]:
data_pool_name = "datapool-filtered-final-version"
data_model_name = "datamodel-filtered-final-version"
activity_table_name = "activities"
rating_table_name = "rating"

[
	DataPool(id='5656d8b6-63fd-41f0-9e10-6183b71028c9', name='datapool-filtered-final-version'),
	DataPool(id='791de0ea-2d8b-43d2-946b-1bc034123683', name='datapool-final-version ')
]

In [5]:
data_pool = celonis.data_integration.get_data_pools().find(data_pool_name)
data_model = data_pool.get_data_models().find(data_model_name)
activity_table = data_model.get_tables().find(activity_table_name)
rating_table = data_model.get_tables().find(rating_table_name)

In [6]:
query = PQL(distinct=False, limit=None, offset=None)
query += PQLColumn(name="Zeit", query=""" "ACTIVITIES"."zeit" """)
query += PQLColumn(name="Vollständiger Name", query=""" "ACTIVITIES"."Vollständiger Name" """)
query += PQLColumn(name="Ereigniskontext - Ereignisname", query=""" "ACTIVITIES"."Ereigniskontext - Ereignisname" """)

activity_df = data_model.export_data_frame(query)

[
	DataModel(id='b405947f-41e9-4abd-a09e-beb990aef1df', name='datamodel-filtered-final-version ', pool_id='5656d8b6-63fd-41f0-9e10-6183b71028c9')
]

In [7]:
query = PQL(distinct=False, limit=None, offset=None)
query += PQLColumn(name="Nr", query=""" "RATING"."Nr" """)
query += PQLColumn(name="Test: Quiz 1 (Punkte)", query=""" "RATING"."Test: Quiz 1 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 2 (Punkte)", query=""" "RATING"."Test: Quiz 2 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 3 (Punkte)", query=""" "RATING"."Test: Quiz 3 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 4 (Punkte)", query=""" "RATING"."Test: Quiz 4 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 5 (Punkte)", query=""" "RATING"."Test: Quiz 5 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 6 (Punkte)", query=""" "RATING"."Test: Quiz 6 (Punkte)" """)
query += PQLColumn(name="Test: Quiz 7 (Punkte)", query=""" "RATING"."Test: Quiz 7 (Punkte)" """)
query += PQLColumn(name="Quiz gesamt (Punkte)", query=""" "RATING"."Quiz gesamt (Punkte)" """)
query += PQLColumn(name="Test: Exam Attempt 1 (Punkte)", query=""" "RATING"."Test: Exam Attempt 1 (Punkte)" """)
query += PQLColumn(name="Kurs gesamt (Punkte)", query=""" "RATING"."Kurs gesamt (Punkte)" """)
query += PQLColumn(name="Name", query=""" "RATING"."Name" """)

rating_df = data_model.export_data_frame(query)

In [8]:
################################################# NEAREST NEIGHBOUR #################################################
"""
# read the data
df = rating_df


df = df.drop(["Quiz gesamt (Punkte)",], axis=1)

# extract X,y arrays
quiz_columns = [string for string in df.columns if "Quiz" in string and "gesamt" not in string]
X = df[quiz_columns].astype(float)
y_test = df["Test: Exam Attempt 1 (Punkte)"].astype(float)
y_total = df["Kurs gesamt (Punkte)"].astype(float)
y_pass = y_total.apply(lambda x: 0 if x < 50 else 1)

# scale X
mmSc = MinMaxScaler()
mmSc.fit(X)
X = mmSc.transform(X)
X = pd.DataFrame(X)
y = y_total

# found KNeighborsRegressor(k) has lowest error with k=2 and LeaveOneOut
# found KNeighborsClassifier(k) has lowest error with k=5 and LeaveOneOut

errors = []
for k in range(1,30):
    collector = []
    # use leave one out, because dataset is so small
    loo = LeaveOneOut()

    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        knn = KNeighborsRegressor(k).fit(X_train, y_train)
        
        y_hat = knn.predict(X_test)
        
        collector.append(y_hat[0])
        
          
    y_pred = np.array(collector)   
    
    errors.append({'k': k,
                  'mae': mean_absolute_error(y, y_pred),
                  'rmse': mean_squared_error(y, y_pred, squared=False)})
    

# store result of lowest error
error_df = pd.DataFrame(errors).round(3)

y = y_pass
errors = []
for k in range(1,30):
    collector = []
    collector_proba = []
    # use leave one out, because dataset is so small
    loo = LeaveOneOut()

    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        knn = KNeighborsClassifier(k).fit(X_train, y_train)
        
        y_hat = knn.predict(X_test)
        y_proba = knn.predict_proba(X_test)[0][1]
        collector.append(y_hat[0])
        collector_proba.append(y_proba)
        
          
    y_pred = np.array(collector)   
    
    errors.append({"k": k,
                    "acc": accuracy_score(y, y_pred),
                    "bacc": balanced_accuracy_score(y, y_pred),
                    "proba": collector_proba})

# store result of lowest error
error_df = pd.DataFrame(errors).round(3)              

proba = error_df['proba'].iloc[error_df['acc'].idxmax()]
"""

[
	DataModelTable(id='043caccb-8ced-4c69-8444-cb0146d3bd2b', data_model_id='b405947f-41e9-4abd-a09e-beb990aef1df', name='rating', alias=None, data_pool_id='5656d8b6-63fd-41f0-9e10-6183b71028c9'),
	DataModelTable(id='159849ab-3520-4fca-97a9-d26b0889e73b', data_model_id='b405947f-41e9-4abd-a09e-beb990aef1df', name='activities', alias=None, data_pool_id='5656d8b6-63fd-41f0-9e10-6183b71028c9')
]

In [9]:
################################################# LINEAR REGRESSION #################################################

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)
df = activity_df

name_counts = df.groupby('Vollständiger Name').size().reset_index(name='activity_count')
activity_count_df = pd.DataFrame({'name': name_counts['Vollständiger Name'], 'activity_count': name_counts['activity_count']})
rating_name_df = rating_df[['Kurs gesamt (Punkte)', 'Name']].copy()
activity_count_df.rename(columns={'Vollständiger Name': 'Name'}, inplace=True)
rating_name_df.rename(columns={'Name': 'name'}, inplace=True)
merged = pd.merge(rating_name_df, activity_count_df, on='name', how='left')
merged.fillna(0.0, inplace=True)

X = merged['activity_count'].values.reshape(-1, 1)
y = merged['Kurs gesamt (Punkte)'].values
reg = LinearRegression().fit(X, y)
y_hat = reg.predict(X).round(3)

pred_lin_act = np.where(y_hat < 50, 0, 1)
points_lin_act = y_hat
plt.scatter(X, y, color='blue', marker='o', label='True Points')
# plt.scatter(X, y_hat, color='red', marker='x', label='Predicted Points')
plt.plot(X, y_hat, color="red")
plt.show()

df = rating_df
X = df['Quiz gesamt (Punkte)'].values.reshape(-1, 1)
y = df['Kurs gesamt (Punkte)'].values
reg = LinearRegression().fit(X, y)
y_hat = reg.predict(X).round(3)

plt.scatter(X, y, color='blue', marker='o', label='True Points')
plt.plot(X, y_hat, color="red")
activity_predict_zero_total = reg.predict([[0.0]])
activity_predict_zero_total = activity_predict_zero_total[0]

pred_lin = np.round(np.where(y_hat < 50, 0, 1), 3)
points_lin = np.round(np.where(y_hat > 0, y_hat, 0), 3)

DataModelTable(id='043caccb-8ced-4c69-8444-cb0146d3bd2b', data_model_id='b405947f-41e9-4abd-a09e-beb990aef1df', name='rating', alias=None, data_pool_id='5656d8b6-63fd-41f0-9e10-6183b71028c9')

In [10]:
################################################# DECISION TREES #################################################

df = rating_df
# extract X,y arrays
quiz_columns = [string for string in df.columns if "Quiz" in string and "gesamt" not in string]
X = df[quiz_columns].astype(float)
y_test = df["Test: Exam Attempt 1 (Punkte)"].astype(float)
y_total = df["Kurs gesamt (Punkte)"].astype(float)
y_pass = y_total.apply(lambda x: 0 if x < 50 else 1)

mmSc = MinMaxScaler()
mmSc.fit(X)
X = mmSc.transform(X)
X = pd.DataFrame(X)
y = y_total


# found DecisionTreeRegressor(max_depth=k), k varies, low and high
errors = []
for i in range(100):
    collector = []
    # use leave one out, because dataset is so small
    loo = LeaveOneOut()

    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        tree_reg = DecisionTreeRegressor().fit(X_train, y_train)
        y_hat = tree_reg.predict(X_test)
        collector.append(y_hat[0])

    y_pred = np.round(np.array(collector), 3)   

    errors.append({'idx': i,
                  'mae': mean_absolute_error(y, y_pred),
                  'rmse': mean_squared_error(y, y_pred, squared=False),
                  "pred": np.where(y_pred < 50, 0, 1),
                  "proba": y_pred})
    
error_df = pd.DataFrame(errors).round(3)
pred_tree = error_df['pred'].iloc[error_df['mae'].idxmax()]
points_tree = error_df['proba'].iloc[error_df['mae'].idxmax()]

DataModelTable(id='043caccb-8ced-4c69-8444-cb0146d3bd2b', data_model_id='b405947f-41e9-4abd-a09e-beb990aef1df', name='rating', alias=None, data_pool_id='5656d8b6-63fd-41f0-9e10-6183b71028c9')

In [11]:
################################################# ADD DECISION TREES RESULT TO RESULT DATA FRAME #################################################

result_df = rating_df
result_df = result_df.sort_values('Nr')
result_df['Abschluss_tree'] = pred_tree
result_df['Punkte_tree'] = points_tree
result_df['Abschluss_lin'] = pred_lin
result_df['Punkte_lin'] = points_lin
result_df['Abschluss_lin_act'] = pred_lin_act
result_df['Punkte_lin_act'] = points_lin_act
result_df = result_df.round(2)

[2023-04-23 10:26:38,245] INFO: Successfully created data export with id 'a81085ad-b315-4387-9b55-b411aef6a6cd'
[2023-04-23 10:26:38,247] INFO: Wait for execution of data export with id 'a81085ad-b315-4387-9b55-b411aef6a6cd'


0it [00:00, ?it/s]

[2023-04-23 10:26:38,385] INFO: Export result chunks for data export with id 'a81085ad-b315-4387-9b55-b411aef6a6cd'
(1102, 3)


,Zeit,Vollständiger Name,Ereigniskontext - Ereignisname
0,2022-09-13 18:48:00,Student1,Test: Quiz 1 - Testversuch gestartet
1,2022-09-13 19:05:00,Student1,Test: Quiz 1 - Testversuch eingereicht
2,2022-09-16 11:47:00,Student1,Test: Quiz 1 - Testversuch gestartet
3,2022-09-16 11:57:00,Student1,Test: Quiz 1 - Testversuch eingereicht
4,2022-09-16 11:57:00,Student1,Test: Quiz 1 - Testversuch gestartet
...,...,...,...
1097,2022-10-19 10:27:00,Student9,Test: Quiz 6 - Testversuch eingereicht
1098,2022-11-07 15:10:00,Student9,Test: Quiz 7 - Testversuch gestartet
1099,2022-11-07 15:31:00,Student9,Test: Quiz 7 - Testversuch eingereicht
1100,2022-11-08 17:51:00,Student9,Test: Exam Attempt 1 - Testversuch gestartet


In [12]:
data_model.reload()

[2023-04-23 10:26:38,773] INFO: Successfully created data export with id '0c826723-34af-4f46-b843-42510ce7ad26'
[2023-04-23 10:26:38,775] INFO: Wait for execution of data export with id '0c826723-34af-4f46-b843-42510ce7ad26'


0it [00:00, ?it/s]

[2023-04-23 10:26:38,861] INFO: Export result chunks for data export with id '0c826723-34af-4f46-b843-42510ce7ad26'
(30, 12)


,Nr,Test: Quiz 1 (Punkte),Test: Quiz 2 (Punkte),Test: Quiz 3 (Punkte),Test: Quiz 4 (Punkte),Test: Quiz 5 (Punkte),Test: Quiz 6 (Punkte),Test: Quiz 7 (Punkte),Quiz gesamt (Punkte),Test: Exam Attempt 1 (Punkte),Kurs gesamt (Punkte),Name
0,0,10.0,11.0,10.0,12.0,8.6,11.0,7.0,46.65,50.0,97.32,Student27
1,1,9.1,10.8,7.7,7.7,0.0,6.1,4.0,30.37,47.5,81.30,Student18
2,3,10.0,11.0,10.0,12.0,13.4,10.5,4.0,46.78,35.0,79.42,Student33
3,4,10.0,11.0,10.0,10.9,13.8,11.0,7.0,49.08,28.0,72.86,Student34
4,5,10.0,11.0,10.0,12.0,14.0,11.0,7.0,50.00,47.5,97.00,Student1
5,6,9.7,11.0,10.0,12.0,14.0,11.0,7.0,49.78,32.0,78.23,Student11
6,7,10.0,11.0,10.0,8.8,0.0,0.0,0.0,25.22,0.0,20.17,Student15
7,8,9.5,11.0,10.0,11.0,13.1,10.7,1.0,43.17,48.0,92.13,Student37
8,9,7.8,9.4,9.1,11.7,10.3,9.0,3.9,40.43,46.7,88.38,Student25
9,10,10.0,11.0,10.0,12.0,14.0,11.0,7.0,50.00,50.0,100.00,Student29
